In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast

from wordcloud import WordCloud, STOPWORDS


from joblib import dump,load
#%matplotlib notebook
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import  make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import random
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor

In [ ]:
onehot = pd.read_csv("/content/drive/MyDrive/Restaurant Rating Prediction/zomato.csv",nrows=20000)

In [ ]:
onehot

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,https://www.zomato.com/bangalore/mudpipe-cafe-...,"1, KDP Building, 4th Floor, Near Forum Mall, A...",Mudpipe Cafe,Yes,Yes,4.5/5,1878,+91 9631220007,BTM,Cafe,"Chaat, Nachos, Peri Peri Pizza, Wedges, Milk S...","Cafe, Continental, Burger",900,"[('Rated 4.0', 'RATED\n Amazing ambience. Goo...","['Crispy Spinach Corn Cheese Stick', 'Alfredo ...",Delivery,Jayanagar
19996,https://www.zomato.com/bangalore/alif-restaura...,"312, 15th cross, 5th Phase, JP Nagar, Bangalore",Alif Restaurant,Yes,No,3.6/5,27,+91 8667307119,JP Nagar,Casual Dining,NaN,"North Indian, Chinese",600,"[('Rated 3.0', ""RATED\n It's a medium size ho...","['Pepper Chicken', 'Grilled Chicken', 'Dum Bri...",Delivery,Jayanagar
19997,https://www.zomato.com/bangalore/black-deer-ko...,"18th Main Park, 17th H Main Road, Koramangala ...",Black Deer,Yes,No,4.4/5,149,080 43751110\r\n+91 9886552002,Koramangala 5th Block,"Takeaway, Delivery","Chocolate Fudge, Chocolate Shake, Bubble Gum S...","Beverages, Desserts",350,"[('Rated 4.0', ""RATED\n I'm not a big fan of ...","['Red Velvet Ice Cream Sundaes', 'Fruit Basket...",Delivery,Jayanagar
19998,https://www.zomato.com/bangalore/supreme-birya...,Delivery Only,Supreme Biryani,Yes,No,NaN,0,+91 9606441698,South Bangalore,"Takeaway, Delivery",NaN,Biryani,400,[],"['Masaledar Egg Biryani [Serves 2]', 'Biryani ...",Delivery,Jayanagar


In [ ]:
onehot.shape

(20000, 17)

In [ ]:
# check for duplicate values
print("No of Duplicates in dataset:  ",onehot.duplicated().sum())
# drop the duplicates
onehot.drop_duplicates(inplace=True)

No of Duplicates in dataset:   0


In [ ]:
# check for duplicate values
print("No of Duplicates in dataset:  ",onehot.duplicated().sum())

No of Duplicates in dataset:   0


In [ ]:
onehot['rate'] = onehot['rate'].replace('NEW',np.NaN) # replace 'NEW' values with NaN
onehot['rate'] = onehot['rate'].replace('-',np.NaN) # replace '-' value with NaN
onehot.dropna(how = 'any', inplace = True) # remove all NaN

onehot['rate'] = onehot.loc[:,'rate'].replace('[ ]','',regex = True) # replace [] with '' string
onehot['rate'] = onehot['rate'].astype(str) # convert to string
onehot['rate'] = onehot['rate'].apply(lambda r: r.replace('/5','')) # replace '/5' character with '' string
onehot['rate'] = onehot['rate'].apply(lambda r: float(r)) # convert string back to float

In [ ]:
onehot.shape

(8912, 17)

In [ ]:
# removed ',' between two words; then converted to categorical features

onehot['cuisines'] = onehot['cuisines'].str.replace(',' , '') # replace ',' with '' string
onehot['cuisines'] = onehot['cuisines'].astype(str).apply(lambda x: ' '.join(sorted(x.split()))) # add white space
onehot['cuisines'].unique() # find unique values

array(['Chinese Indian Mughlai North', 'Chinese Indian North Thai',
       'Cafe Italian Mexican', ..., 'Afghan Indian North',
       'Bakery Cafe Italian', 'Beverages Fast Food Food Healthy'],
      dtype=object)

In [ ]:
# removed ',' between two words; then converted to categorical features

onehot['rest_type'] = onehot['rest_type'].str.replace(',' , '') 
onehot['rest_type'] = onehot['rest_type'].astype(str).apply(lambda x: ' '.join(sorted(x.split())))
onehot['rest_type'].value_counts().head()

Casual Dining        2858
Bites Quick          2012
Cafe                  897
Bar Casual Dining     520
Dessert Parlor        397
Name: rest_type, dtype: int64

In [ ]:
# removed ',' between two words; then converted to categorical features

onehot['dish_liked'] = onehot['dish_liked'].str.replace(',' , '') 
onehot['dish_liked'] = onehot['dish_liked'].astype(str).apply(lambda x: ' '.join(sorted(x.split())))
onehot['dish_liked'].value_counts().head()

Biryani            63
Biryani Chicken    30
Waffles            27
Friendly Staff     25
Dosa Masala        22
Name: dish_liked, dtype: int64

In [ ]:
# removed ',' between two words; then converted string to int value

onehot['approx_cost(for two people)'] = onehot['approx_cost(for two people)'].str.replace(',' , '') 
onehot.rename(columns={'approx_cost(for two people)': 'average_cost'}, inplace=True)

In [ ]:
x = onehot.drop(['rate','name'],axis = 1)
y = onehot['rate']

In [ ]:
train_data,test_data,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 33)

In [ ]:
dump(train_data, '/content/drive/MyDrive/Restaurant Rating Prediction/without_featurization/train_data')
dump(test_data, '/content/drive/MyDrive/Restaurant Rating Prediction/without_featurization/test_data')
dump(y_train, '/content/drive/MyDrive/Restaurant Rating Prediction/without_featurization/y_train')
dump(y_test, '/content/drive/MyDrive/Restaurant Rating Prediction/without_featurization/y_test')

['/content/drive/MyDrive/Restaurant Rating Prediction/without_featurization/y_test']

In [ ]:
train_data.shape

(6238, 15)

In [ ]:
test_data.shape

(2674, 15)

In [ ]:
train_data = load('/content/drive/MyDrive/Restaurant Rating Prediction/without_featurization/train_data')
test_data = load('/content/drive/MyDrive/Restaurant Rating Prediction/without_featurization/test_data')
y_train = load('/content/drive/MyDrive/Restaurant Rating Prediction/without_featurization/y_train')
y_test = load('/content/drive/MyDrive/Restaurant Rating Prediction/without_featurization/y_test')

In [ ]:
all_features =[]
# test_feature=[]

Response Coded Features

In [ ]:
# re-insert rate column in train data
train_data.insert(2, 'Rate', y_train, allow_duplicates = False)

In [ ]:
# check shape
train_data.shape

(6238, 16)

In [ ]:
train_data.head(2)

,url,address,Rate,online_order,book_table,votes,phone,location,rest_type,dish_liked,cuisines,average_cost,reviews_list,menu_item,listed_in(type),listed_in(city)
14899,https://www.zomato.com/bangalore/dolci-dessert...,"18/3, Ali Askar Road, Near Kingfisher Airlines...",4.3,Yes,No,654,080 40977132,Cunningham Road,Cafe Dessert Parlor,Blueberry Cappuccino Cheesecake Cheesecake Cof...,Cafe Desserts,750,"[('Rated 4.0', 'RATED\n A cozy little place w...","['Nutella Ferrero Rocher', 'Dark chocolate Fud...",Delivery,Frazer Town
2254,https://www.zomato.com/bangalore/manis-dum-bir...,"471, Kothanur Dinne Main Road, 1st Floor, RBI ...",4.2,Yes,No,374,+91 9606441698,JP Nagar,Casual Dining,Biryani Biryani Biryani Biryani Biryani Bonele...,Biryani Kebab,750,"[('Rated 3.0', 'RATED\n Quality Biriyani but ...",[],Dine-out,Bannerghatta Road


In [ ]:
# Re-Insert 'rate' column in test dataframe.
test_data.insert(2, 'Rate', y_test, allow_duplicates = False)
test_data.shape

(2674, 16)

In [ ]:
test_data.head(2)

,url,address,Rate,online_order,book_table,votes,phone,location,rest_type,dish_liked,cuisines,average_cost,reviews_list,menu_item,listed_in(type),listed_in(city)
12886,https://www.zomato.com/bangalore/nisarga-garde...,"147, Opposite Police Commissioner's Office, In...",4.0,No,No,89,080 22289563,Infantry Road,Bites Quick,Akki Coffee Dosa Idli Masala Raagi Rava Roti R...,Chinese Food Indian Juices South Street,300,"[('Rated 3.5', 'RATED\n Tucked in the old wor...",[],Dine-out,Church Street
18597,https://www.zomato.com/bangalore/kantis-thippa...,"70, 1st Cross, New Thippasandra, Thippasandra,...",3.3,Yes,No,66,080 25296559\n+91 8296359142,Thippasandra,Shop Sweet,Paratha Thali,Food Mithai Street,300,"[('Rated 5.0', ""RATED\n A whole lot of variet...",[],Desserts,Indiranagar


In [ ]:
# # https://www.geeksforgeeks.org/python-creating-a-pandas-dataframe-column-based-on-a-given-condition/

key_dict = dict()
def provide_response_coded_features(groupByVal,columnName, df):
    
    '''
    This function is used to convert categorical features into response coded features.
    It simply perform MEAN VALUE REPLACEMENT.
    '''
    
    ## distribute values group by n take mean of rate column
    mean_df = df.groupby([groupByVal]).mean()
    ## stored in dict data type.. key is column name and values in mean value of rate column.
    mean_dict =mean_df['Rate'].to_dict()
    key_dict.update([ (groupByVal, mean_dict) ] )
    for k, v in mean_dict.items():
            mean_dict[k] = round(v,2)
    df[columnName] = df[groupByVal].map(mean_dict) 
    return df

In [ ]:
# create response coded feature for online_order feature.
mean_online_order =provide_response_coded_features('online_order','mean_online_order',train_data)
mean_online_order[['Rate','online_order','mean_online_order']][:10]

,Rate,online_order,mean_online_order
14899,4.3,Yes,3.87
2254,4.2,Yes,3.87
9034,3.8,Yes,3.87
1059,3.0,Yes,3.87
15402,3.9,Yes,3.87
17817,3.4,Yes,3.87
778,3.8,No,3.90
1045,3.9,Yes,3.87
6605,4.4,No,3.90
3234,4.2,Yes,3.87


In [ ]:
# create response coded feature for book_table feature.

mean_book_table =provide_response_coded_features('book_table','mean_book_table',train_data)
mean_book_table[['Rate','book_table','mean_book_table']][:10]

,Rate,book_table,mean_book_table
14899,4.3,No,3.79
2254,4.2,No,3.79
9034,3.8,No,3.79
1059,3.0,No,3.79
15402,3.9,No,3.79
17817,3.4,No,3.79
778,3.8,No,3.79
1045,3.9,No,3.79
6605,4.4,No,3.79
3234,4.2,No,3.79


In [ ]:
# create response coded feature for rest_type feature.

mean_rest_type =provide_response_coded_features('rest_type','mean_rest_type',train_data)
mean_rest_type[['Rate','rest_type','mean_rest_type']][:20]

,Rate,rest_type,mean_rest_type
14899,4.3,Cafe Dessert Parlor,4.17
2254,4.2,Casual Dining,3.82
9034,3.8,Casual Dining,3.82
1059,3.0,Bites Quick,3.72
15402,3.9,Casual Dining,3.82
17817,3.4,Casual Dining,3.82
778,3.8,Cafe Casual Dining,4.14
1045,3.9,Casual Dining,3.82
6605,4.4,Casual Dining Microbrewery,4.35
3234,4.2,Dessert Parlor,4.07


In [ ]:
# create response coded feature for location feature.

mean_location =provide_response_coded_features('location','mean_location',train_data)
mean_location[['Rate','location','mean_location']][:10]

,Rate,location,mean_location
14899,4.3,Cunningham Road,4.07
2254,4.2,JP Nagar,3.85
9034,3.8,Jayanagar,3.96
1059,3.0,JP Nagar,3.85
15402,3.9,Frazer Town,3.82
17817,3.4,Indiranagar,3.94
778,3.8,Basavanagudi,3.85
1045,3.9,Bannerghatta Road,3.69
6605,4.4,Lavelle Road,4.17
3234,4.2,Jayanagar,3.96


In [ ]:
# create response coded feature for cuisines feature.

mean_cuisines =provide_response_coded_features('cuisines','mean_cuisines',train_data)
mean_cuisines[['Rate','cuisines','mean_cuisines']][:10]

,Rate,cuisines,mean_cuisines
14899,4.3,Cafe Desserts,4.12
2254,4.2,Biryani Kebab,3.88
9034,3.8,Mangalorean Seafood,4.04
1059,3.0,Chinese Food Indian North Street,3.64
15402,3.9,Biryani Indian Mughlai North,3.77
17817,3.4,Andhra Indian Indian North South,3.47
778,3.8,Cafe Chinese Fast Food Indian North,3.80
1045,3.9,Chinese Indian North,3.53
6605,4.4,Indian Modern,4.39
3234,4.2,Bakery Desserts,3.95


In [ ]:
# create response coded feature for dish_liked feature.

mean_dish_liked =provide_response_coded_features('dish_liked','mean_dish_liked',train_data)
mean_dish_liked[['Rate','dish_liked','mean_dish_liked']][:10]

,Rate,dish_liked,mean_dish_liked
14899,4.3,Blueberry Cappuccino Cheesecake Cheesecake Cof...,4.3
2254,4.2,Biryani Biryani Biryani Biryani Biryani Bonele...,4.2
9034,3.8,Biryani Dosa Fish Food Ghee Murgh Neer Roast S...,3.8
1059,3.0,Aloo Amritsari Bhature Chole Dal Kheer Kulcha ...,3.0
15402,3.9,Bheja Biryani Brain Chicken Chicken Fry Grill ...,3.9
17817,3.4,Bhurji Biryani Chicken Coffee Dosa Egg Grain H...,3.4
778,3.8,Chilli Idli Lime Noodles Paratha Sandwich Sand...,3.8
1045,3.9,Chicken Chicken Chicken Chicken Fish Grill Hyd...,3.9
6605,4.4,Appletini Beer Cocktails Lamb Martini Sangria ...,4.4
3234,4.2,Burgers Cake Chocolate Chocolava Mousse Puff R...,4.2


In [ ]:
for feature,values in key_dict.items():
    print(feature)

online_order
book_table
rest_type
location
cuisines
dish_liked


In [ ]:
def return_dict_mean_value(query_feature):
    
    '''
    'key_dict' is dictionary object which has all the Categorical variable names store as KEY and its mean as VALUE.
    This is function is used to return mean value for query_feature.
    
    KEY ==> 
    Value ==> Mean value response to that key
    
    query_feature ==>  Desired key
    Return ==> Categorical feature and their corresponding mean values. 
    '''
    
    result_dict=dict()

    for feature_name, values in key_dict.items():    
        if feature_name == query_feature:
            for key in values:
                result_dict.update([ (key, values[key]) ] )

                print(key + ':', values[key])
    return result_dict
return_dict_mean_value('online_order')

No: 3.9
Yes: 3.87


{'No': 3.9, 'Yes': 3.87}

In [ ]:
## similarly we will create same response coded features for test dataset
## Test data

dict_online = return_dict_mean_value('online_order')
dict_book_table = return_dict_mean_value('book_table')
dict_rest_type = return_dict_mean_value('rest_type')
dict_location = return_dict_mean_value('location')
dict_cuisines = return_dict_mean_value('cuisines')
dict_dish_liked = return_dict_mean_value('dish_liked')

No: 3.9
Yes: 3.87
No: 3.79
Yes: 4.14
Bakery: 3.91
Bakery Bites Quick: 3.61
Bakery Cafe: 4.0
Bakery Dessert Parlor: 4.04
Bar: 3.93
Bar Bites Quick: 4.0
Bar Cafe: 4.1
Bar Casual Dining: 4.16
Bar Dining Fine: 4.16
Bar Lounge: 3.91
Bar Microbrewery: 4.4
Bar Pub: 4.15
Beverage Bites Quick Shop: 3.96
Beverage Cafe Shop: 3.9
Beverage Dessert Parlor Shop: 3.99
Beverage Shop: 4.08
Bites Cafe Quick: 3.57
Bites Court Food Quick: 3.74
Bites Dessert Parlor Quick: 3.73
Bites Quick: 3.72
Bites Quick Shop Sweet: 3.9
Cafe: 3.95
Cafe Casual Dining: 4.14
Cafe Dessert Parlor: 4.17
Cafe Lounge: 4.14
Cafe Pub: 4.7
Cafee Casual Dining Irani: 4.4
Casual Club Dining: 4.0
Casual Dining: 3.82
Casual Dining Lounge: 4.07
Casual Dining Microbrewery: 4.35
Casual Dining Pub: 4.13
Club: 4.05
Court Food: 3.69
Delivery: 3.78
Delivery Takeaway: 3.78
Dessert Kiosk Parlor: 2.9
Dessert Parlor: 4.07
Dining Fine: 4.2
Dining Fine Lounge: 4.4
Dining Fine Microbrewery: 4.3
Food Truck: 3.63
Kiosk: 3.97
Lounge: 3.97
Lounge Microbr

In [ ]:
test_data['mean_online_order'] = test_data['online_order'].map(dict_online) 

In [ ]:
test_data['mean_book_table'] = test_data['book_table'].map(dict_book_table) 

In [ ]:
test_data['mean_rest_type'] = test_data['rest_type'].map(dict_rest_type) 

In [ ]:
test_data['mean_location'] = test_data['location'].map(dict_location) 

In [ ]:
test_data['mean_cuisines'] = test_data['cuisines'].map(dict_cuisines) 

In [ ]:
test_data['mean_dish_liked'] = test_data['dish_liked'].map(dict_dish_liked) 

In [ ]:
##check NaN values. NaN value arise because there are some categories those are not present in test set.
test_data.isna().sum()

url                    0
address                0
Rate                   0
online_order           0
book_table             0
votes                  0
phone                  0
location               0
rest_type              0
dish_liked             0
cuisines               0
average_cost           0
reviews_list           0
menu_item              0
listed_in(type)        0
listed_in(city)        0
mean_online_order      0
mean_book_table        0
mean_rest_type         0
mean_location          3
mean_cuisines         94
mean_dish_liked      511
dtype: int64

In [ ]:
print("There are some category which is not present in train set which is % ",((2+64+489)/test_data.shape[0])*100)

There are some category which is not present in train set which is %  20.75542258788332


It is not large number so we can simply neglect those data points.

In [ ]:
# drop null values
test_data.dropna(subset=['mean_dish_liked','mean_rest_type','mean_cuisines'],inplace=True)

In [ ]:
test_data.isna().sum()

url                  0
address              0
Rate                 0
online_order         0
book_table           0
votes                0
phone                0
location             0
rest_type            0
dish_liked           0
cuisines             0
average_cost         0
reviews_list         0
menu_item            0
listed_in(type)      0
listed_in(city)      0
mean_online_order    0
mean_book_table      0
mean_rest_type       0
mean_location        0
mean_cuisines        0
mean_dish_liked      0
dtype: int64

In [ ]:
y_test= test_data['Rate']

Delete Redudant columns¶

In [ ]:
train_data.columns

Index(['url', 'address', 'Rate', 'online_order', 'book_table', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'average_cost', 'reviews_list', 'menu_item', 'listed_in(type)',
       'listed_in(city)', 'mean_online_order', 'mean_book_table',
       'mean_rest_type', 'mean_location', 'mean_cuisines', 'mean_dish_liked'],
      dtype='object')

In [ ]:
train_data.drop(columns=['rest_type','location','cuisines','dish_liked','menu_item','url','phone','reviews_list','listed_in(type)','listed_in(city)','address','online_order','book_table','Rate'], inplace=True)

In [ ]:
train_data.columns

Index(['votes', 'average_cost', 'mean_online_order', 'mean_book_table',
       'mean_rest_type', 'mean_location', 'mean_cuisines', 'mean_dish_liked'],
      dtype='object')

In [ ]:
test_data.columns

Index(['url', 'address', 'Rate', 'online_order', 'book_table', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'average_cost', 'reviews_list', 'menu_item', 'listed_in(type)',
       'listed_in(city)', 'mean_online_order', 'mean_book_table',
       'mean_rest_type', 'mean_location', 'mean_cuisines', 'mean_dish_liked'],
      dtype='object')

In [ ]:
test_data.drop(columns=['rest_type','location','cuisines','dish_liked','menu_item','url','phone','reviews_list','listed_in(type)','listed_in(city)','address','online_order','book_table','Rate'], inplace=True)

In [ ]:
test_data.columns

Index(['votes', 'average_cost', 'mean_online_order', 'mean_book_table',
       'mean_rest_type', 'mean_location', 'mean_cuisines', 'mean_dish_liked'],
      dtype='object')

Checking Shape

In [ ]:
# always verify shapes
train_data.shape, y_train.shape

((6238, 8), (6238,))

In [ ]:
# always verify shapes
test_data.shape, y_test.shape

((2153, 8), (2153,))

Model - 1 Linear Regression Model

In [ ]:
from sklearn.metrics import  make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn import metrics


def mse(y, y_pred):
    return np.mean((y_pred - y)**2) 

mse_scorer = make_scorer(mse, greater_is_better=False)

In [ ]:
lr = LinearRegression()
lr.fit(train_data,y_train)
y_pred_lr = lr.predict(test_data)

mse(y_test, y_pred_lr)

0.010519191018807567

feature Importance¶

Output Visualization

In [ ]:
Randpred = pd.DataFrame({ "actual": y_test, "pred": y_pred_lr })
Randpred

,actual,pred
12886,4.0,3.996010
18597,3.3,3.607620
1203,3.9,3.899686
2251,4.1,4.105914
13449,4.0,4.002857
...,...,...
10579,4.0,3.990027
14955,4.2,4.209982
15209,3.7,3.696638
7764,3.5,3.493244


Model - 2 SGD Regression Model

In [ ]:
sgdReg = linear_model.SGDRegressor()
sgdReg.fit(train_data,y_train)
y_pred_sgdr = sgdReg.predict(test_data)

mse(y_test, y_pred_sgdr)

1.9069293660985524e+28

No need to find feature importance, model is not learning.

Model - 3 Random Forest Regressor Model

In [ ]:
rfr = RandomForestRegressor()
rfr.fit(train_data,y_train)
y_pred_rfr = rfr.predict(test_data)

mse(y_test, y_pred_rfr)

0.005113818808432837

In [ ]:

import pickle

In [ ]:
with open('/content/drive/MyDrive/Restaurant Rating Prediction/model_pickle','wb') as f:
  pickle.dump(rfr,f)

In [ ]:
with open('/content/drive/MyDrive/Restaurant Rating Prediction/model_pickle','rb') as f:
  map = pickle.load(f)

In [ ]:
map

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=250, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
map.predict(test_data.values)

array([4.    , 3.6252, 3.9   , ..., 3.7   , 3.4812, 4.1   ])

Hyperparam Tuning

In [ ]:
tuned_parameters = {'n_estimators': [250,500]}

grd_regressor = GridSearchCV(RandomForestRegressor(), tuned_parameters, cv=10, 
                   n_jobs=-1, verbose=1, scoring=mse_scorer)
grd_regressor.fit(train_data, y_train)

Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   59.6s finished


GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_j

In [ ]:
print(grd_regressor.best_params_)

{'n_estimators': 250}


Best Model "Random forest Regreesion"

In [ ]:

rfr = RandomForestRegressor(n_estimators=500)
rfr.fit(train_data,y_train)
y_pred_rfr = rfr.predict(test_data)

mse(y_test, y_pred_rfr)

0.005205715601805845

In [ ]:
rfr = RandomForestRegressor(n_estimators=250)
rfr.fit(train_data,y_train)
y_pred_rfr1 = rfr.predict(test_data)

mse(y_test, y_pred_rfr1)

0.005251476037104011

Output Visualization¶

In [ ]:
Randpred = pd.DataFrame({ "actual": y_test, "pred": y_pred_rfr })
Randpred

,actual,pred
12886,4.0,4.0000
18597,3.3,3.6244
1203,3.9,3.9000
2251,4.1,4.1000
13449,4.0,4.0000
...,...,...
10579,4.0,4.0000
14955,4.2,4.2000
15209,3.7,3.7000
7764,3.5,3.4668


In [ ]:
from prettytable import PrettyTable


x = PrettyTable()
x.field_names =  ['Model with response coded features','MSE']

x.add_row(["Linear Regression ", 0.01051])
x.add_row(["SGD Regressor ", 1.9069])
x.add_row(["Random Forest Regressor", 0.0051])
x.add_row(["Hyperparam Tuning on Random Forest Regressor", 0.0052])





print('\n')
print(x)



+----------------------------------------------+---------+
|      Model with response coded features      |   MSE   |
+----------------------------------------------+---------+
|              Linear Regression               | 0.01051 |
|                SGD Regressor                 |  1.9069 |
|           Random Forest Regressor            |  0.0051 |
| Hyperparam Tuning on Random Forest Regressor |  0.0052 |
+----------------------------------------------+---------+


Here we can conclude that **Random Forest Regressor** is best model with lowest **MSE = 0.0051**